In [653]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from agoda_cancellation_estimator import AgodaCancellationEstimator
from agoda_cancellation_prediction import load_data
import pandas as pd
import numpy as np


In [654]:
full_data = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\agoda_cancellation_train.csv").fillna(False)
test_data_w1 = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\test_set_week_1.csv").dropna().drop_duplicates()
full_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09 00:00:00,2018-07-13 00:00:00,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,...,False,False,False,False,False,False,1192,False,False,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16 00:00:00,2018-08-17 00:00:00,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,False,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14 00:00:00,2018-09-15 00:00:00,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,False,3819,False,False,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02 00:00:00,2018-07-03 00:00:00,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,False,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09 00:00:00,2018-08-10 00:00:00,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,...,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,False,False,616


In [655]:

full_data.accommadation_type_name = full_data.accommadation_type_name.astype('category')
full_data.customer_nationality = full_data.customer_nationality.astype(str).astype('category')
#full_data.hotel_id = full_data.hotel_id.astype('category')
full_data.charge_option = full_data.charge_option.astype('category')
#full_data.booking_datetime = full_data.booking_datetime.astype('category')
full_data.cancellation_policy_code = full_data.cancellation_policy_code.astype('category')
full_data['customer_nationality_codes'] = full_data.customer_nationality.cat.codes
full_data['accommadation_type_name_codes'] = full_data.accommadation_type_name.cat.codes
#full_data['hotel_id_codes'] = full_data.hotel_id.cat.codes
full_data['charge_option_codes'] = full_data.charge_option.cat.codes
full_data['cancellation_policy_code_codes'] = full_data.cancellation_policy_code.cat.codes
b1 = full_data['booking_datetime'].aggregate(pd.Timestamp)
b2 = full_data['checkin_date'].aggregate(pd.Timestamp)
b2 = pd.to_datetime(full_data['checkin_date']).apply(lambda x: x.date())
b1 = pd.to_datetime(full_data['booking_datetime']).apply(lambda x: x.date())

full_data['days_before_checkin'] = [i.days for i in (b2.to_numpy() - b1.to_numpy())]


In [656]:
(pd.Timestamp('2020-03-02')-pd.Timestamp('2020-02-01')).days

30

In [657]:
dates = {"2018-07-17", "2018-07-18", "2018-07-19", "2018-07-20", "2018-07-21", "2018-07-22", "2018-07-23"}
# full_data['label'] = full_data.cancellation_datetime.map(lambda x: bool(x) & True & (x in dates))
full_data['label'] = full_data.cancellation_datetime.map(lambda x: bool(x) & True)
# full_data['label'] = full_data.cancellation_datetime.map(lambda x: x > '2018-08-15' and x < '2018-08-20')
full_data.loc[:, ['cancellation_datetime', 'label']]

,cancellation_datetime,label
0,False,False
1,False,False
2,False,False
3,False,False
4,2018-07-23,True
...,...,...
58654,2018-07-22,True
58655,False,False
58656,False,False
58657,False,False


In [658]:
# split the data into training set and test set
train_ratio = 0.75
rand_nums = np.random.random_sample(len(full_data))
full_data['train'] =  rand_nums <= train_ratio
full_data.train.value_counts()
train_df = full_data[full_data.train]
test_df = full_data[~full_data.train]
len(train_df), len(test_df), len(full_data)

(43939, 14720, 58659)

In [659]:
features_cols = ["hotel_id", "days_before_checkin",
                "accommadation_type_name_codes",
                 "charge_option_codes",
                "hotel_star_rating",
                "customer_nationality_codes",
                 "cancellation_policy_code_codes"]
train_features = train_df[features_cols]
test_features = test_df[features_cols]
train_features.head()


,hotel_id,days_before_checkin,accommadation_type_name_codes,charge_option_codes,hotel_star_rating,customer_nationality_codes,cancellation_policy_code_codes
2,780431,0,10,1,4.0,115,245
3,291365,7,10,0,3.0,110,151
5,59272,110,16,0,3.0,127,694
6,400966,3,19,1,3.5,121,152
7,69287,23,10,0,3.5,120,152


In [660]:
# features = full_data[[#"h_booking_id",
#                       "hotel_id",
#                       "accommadation_type_name_codes",
#                       "hotel_star_rating",
#                       "customer_nationality_codes"]]
# features.head()


In [661]:
train_labels = train_df["label"]
test_labels = test_df["label"]
train_labels.head()

2    False
3    False
5     True
6    False
7     True
Name: label, dtype: bool

In [662]:
full_data['cancellation_datetime']

0             False
1             False
2             False
3             False
4        2018-07-23
            ...    
58654    2018-07-22
58655         False
58656         False
58657         False
58658         False
Name: cancellation_datetime, Length: 58659, dtype: object

In [663]:
random_classifier = RandomForestClassifier()
random_forest_regressor = RandomForestRegressor()

In [664]:
y_train = train_df.label.values
X_train = train_features.values
y_test = test_df.label.values
X_test = test_features.values
X_train

array([[7.804310e+05, 0.000000e+00, 1.000000e+01, ..., 4.000000e+00,
        1.150000e+02, 2.450000e+02],
       [2.913650e+05, 7.000000e+00, 1.000000e+01, ..., 3.000000e+00,
        1.100000e+02, 1.510000e+02],
       [5.927200e+04, 1.100000e+02, 1.600000e+01, ..., 3.000000e+00,
        1.270000e+02, 6.940000e+02],
       ...,
       [3.579174e+06, 1.000000e+00, 0.000000e+00, ..., 4.000000e+00,
        1.000000e+02, 6.150000e+02],
       [4.071560e+05, 6.200000e+01, 1.000000e+01, ..., 5.000000e+00,
        2.500000e+01, 3.400000e+02],
       [2.227632e+06, 5.000000e+00, 1.000000e+01, ..., 4.000000e+00,
        2.500000e+01, 3.400000e+02]])

In [665]:
model = random_classifier.fit(X_train,y_train)
model2 = random_forest_regressor.fit(X_train,y_train)

In [666]:
# how many orders the model predicted were cancelled, on a data set he was trained on
#np.bincount(model.predict(X_train))


In [667]:
#np.bincount(y_train)

In [668]:
#model.predict(X_test)

In [669]:
# how many orders the model predicted were cancelled, on a data set he was not trained on
#np.bincount(model.predict(X_test))

In [670]:
# how many orders got actually cancelled
np.bincount(y_test)

array([10841,  3879], dtype=int64)

In [671]:
# how many orders that got cancelled, the model guessed were cancelled
np.bincount(model.predict(X_test) & y_test)

array([12547,  2173], dtype=int64)

In [672]:
#np.bincount(model.predict(X_train) & y_train)

In [673]:
#full_data.label.value_counts()

In [674]:
#full_data[(full_data.cancellation_datetime > '2018-08-07') & (full_data.cancellation_datetime < '2018-08-20')]

In [675]:
test_data_w1 = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\test_set_week_1.csv").dropna().drop_duplicates()
test_data_w1.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
2,-9217919187333201744,2018-11-25 21:44:00,2019-04-22 00:00:00,2019-04-23 00:00:00,400322,JP,2012-10-02 13:54:00,3.0,Hotel,Pay Later,...,0.0,0.0,0.0,1.0,0.0,0.0,2884,193.0,344.0,2264
3,-9215699014988905507,2018-11-20 20:20:00,2018-12-16 00:00:00,2018-12-20 00:00:00,3135771,PH,2017-11-06 17:05:00,5.0,Resort,Pay Now,...,0.0,1.0,1.0,0.0,1.0,0.0,2532,415.0,291.0,1405
5,-9209328318584438243,2018-11-07 18:33:00,2018-12-21 00:00:00,2018-12-22 00:00:00,10602,SG,2014-05-20 08:05:00,5.0,Hotel,Pay Now,...,0.0,0.0,0.0,1.0,0.0,0.0,2868,870.0,551.0,437
10,-9206569785675213125,2018-11-18 12:54:00,2018-12-28 00:00:00,2018-12-29 00:00:00,2559374,HK,2017-07-20 15:36:00,4.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,0.0,1633,613.0,355.0,142
27,-9162547391520652865,2018-11-17 21:38:00,2018-12-31 00:00:00,2019-01-02 00:00:00,185040,JP,2010-05-03 08:59:00,3.0,Hotel,Pay Later,...,0.0,0.0,0.0,0.0,0.0,0.0,4463,658.0,50.0,2567


In [676]:
# convert accomodation type name to codes